In [13]:
cd ../src

/gpfs/home/gdb20/code/batch-manager/src


In [14]:
import input_files
import input_combi
import helpers
import os

In [15]:
global_configs = { '' : {
        'num_cores' : 8,
        'mem_per_cpu_GB' : 4,
        'runtime' : '0-00:20:00',
        #'!max_jobs' : 10, #this doesn't do anything yet
        #'solvent' : 'thf', #this is what Mike told me we use
    }
}

In [16]:
xyz_root = '/gpfs/home/gdb20/code/mol-maker/data/mike/nicstest'
mike_dir = os.path.join(xyz_root,'')

In [17]:

mike_mols = input_combi.xyz_files_from_directory(mike_dir)
mike_mols['!directories'] = True

In [18]:


orbital_and_uno_occs_blocks = {
    'output' : [
        'print[P_UNO_OccNum] 1'
    ],    
    'plots'  : [
        'dim1 100',
        'dim2 100',
        'dim3 100',
        'Format Gaussian_Cube',
        'SpinDens("spindensity");',
        'ElDens("electrondensity");',
    ]
}   

tddft_block = {
    'tddft' : [
        'maxdim 5',
        'nroots 50',
        'DoNTO true',
        'NTOStates 1,2,3,4,5,6,7,8,9,10', #somewhat arbitrary choice
    ],
}


orca_defaults = {
    'program' : 'ORCA',
    'integration_grid' : 'DEFGRID2', #default settings
    'scf_tolerance' : 'TightSCF', #see if they work in ORCA 6
}

orca_singlepoint = orca_defaults.copy()
orca_singlepoint.update({      
    'natural_orbitals' : True,
    'runtime' : '0-05:00:00',
    'blocks' : orbital_and_uno_occs_blocks,   
    'run_type' : None,
})

orca_opt_freq = orca_defaults.copy()
orca_opt_freq.update({
    'runtime' : '3-00:00:00',
    'run_type' : 'OPT FREQ',
})

orca_tddft = orca_defaults.copy()
orca_tddft.update({
    'runtime' : '4-00:00:00',
    'run_type' : None,
    'blocks' : tddft_block,
})


orca_nics = orca_defaults.copy()
orca_nics.update({
    'runtime' : '2-00:00:00',
    'run_type' : 'NMR', #GIAO is the default for ORCA
})

gaussian_nics = {
    'program'  : 'Gaussian',
    'runtime'  : '2-00:00:00',
    'run_type' : 'NMR=GIAO', 
}

gaussian_nics_preprocessing = {
    'num_cores' : 1,
    'mem_per_cpu_GB' : 3,
    'runtime' : '0-00:05:00',
    'program'  : 'pyAroma',
    'cc_program' : 'Gaussian',
}

orca_nics_preprocessing = {
    'num_cores' : 1,
    'mem_per_cpu_GB' : 3,
    'runtime' : '0-00:05:00',
    'program'  : 'pyAroma',
    'cc_program' : 'ORCA',
}


r2scan_3c = {
    'functional' : 'r2SCAN-3c',    
    'basis' : None,
    'aux_basis' : None,
}

cam_b3lyp = {
    'functional' : 'CAM-B3LYP',
    'basis' : '6-31G(d,p)',
    'aux_basis' : 'autoaux',
}
cam_b3lyp_diffuse = cam_b3lyp.copy()
cam_b3lyp_diffuse.update({'basis': '6-31G(d,p)'})

wb97x_3c = {
    'functional' : 'wB97X-3c',
    'basis_set' : None,
    'aux_basis' : None,
}
neutral_singlet = {
    'charge' : 0,
    'spin_multiplicity' : 1,
    'broken_symmetry' : False,
}

neutral_triplet = {
    'charge' : 0,
    'spin_multiplicity' : 3,
    'broken_symmetry' : False,
}

neutral_singlet_brokensym = {
    'charge' : 0,
    'spin_multiplicity' : 3,
    'broken_symmetry' : True,
}

neutral_singlet_guessmix = {
    'charge' : 0,
    'spin_multiplicty' : 1,
    'uks' : True,
    'keywords' : ['guess=mix']
}

crest_quick_settings = {
    'program' : 'CREST',
    'functional' : 'gfn2',
    'quick' : 'quick',
    'reopt' : True,
    'cluster' : True,
    'noreftopo' : True, #might regret this
}

xtb_cm_states = {
    '0_1' : neutral_singlet,
    '0_3': neutral_triplet,
}
orca_cm_states = xtb_cm_states.copy()
orca_cm_states.update({
    '0_1' : neutral_singlet_brokensym,
})

gaussian_cm_states = xtb_cm_states.copy()
gaussian_cm_states.update({
    '0_1' : neutral_singlet_guessmix,
})


In [22]:
crest_gfn2__gfnff_squick = crest_quick_settings.copy()
crest_gfn2__gfnff_squick.update({        
    'functional' : 'gfn2//gfnff',
    'quick' : 'squick',
})

#SINGLEPOINTS
r2scan_3c_triplet_sp = orca_singlepoint.copy()
r2scan_3c_triplet_sp.update({          
        **r2scan_3c,
        **neutral_triplet,   
        '!coords_from' : '../xtb_gfn2_opt_freq_test',
        '!xyz_file' : 'xtbopt.xyz',
        #'!overwrite' : True, #VERY DANGEROUS, comment out when not using     
})

r2scan_3c_singlet_sp = orca_singlepoint.copy()
r2scan_3c_singlet_sp.update({ 
        **r2scan_3c,
        **neutral_singlet_brokensym,
        '!coords_from' : '../xtb_gfn2_opt_freq_test',
        '!xyz_file' : 'xtbopt.xyz',
        #'!overwrite' : True,  #VERY DANGEROUS, comment out when done using  
})

r2scan_3c_tddft = orca_tddft.copy() #let's only do one of these
r2scan_3c_tddft.update({
    **r2scan_3c,
    '!coords_from' : '../xtb_gfn2_opt_freq_test',
    '!xyz_file' : 'xtbopt.xyz',
})

########JOBS FOR DOING NICS
gauss_nics_prepro = gaussian_nics_preprocessing.copy()
gauss_nics_prepro.update({
    'xyz_file' :  'xtbopt.xyz',
    '!coords_from' : '../xtb_gfn2_opt_freq_test',
    '!xyz_file' : 'xtbopt.xyz',
    '!overwrite' : True, 
})

orca_nics_prepro = orca_nics_preprocessing.copy()
orca_nics_prepro.update({
    'xyz_file' :  'xtbopt.xyz',
    '!coords_from' : '../xtb_gfn2_opt_freq_test',
    '!xyz_file' : 'xtbopt.xyz',
    '!overwrite' : True, 
})

cam_b3lyp_6_31_plus_gdp_NICS_gaussian = gaussian_nics.copy()
cam_b3lyp_6_31_plus_gdp_NICS_gaussian.update({
    **cam_b3lyp_diffuse,
    '!coords_from' : '../gaussian_prepro_test',
    '!xyz_file'    : 'gaussian_prepro_test.xyz', #I don't like the way this works. will change
    '!overwrite' : True, 
})

cam_b3lyp_6_31_plus_gdp_NICS_gaussian.update({
    'aux_basis' : None,
})

cam_b3lyp_6_31_plus_gdp_NICS = orca_nics.copy()
cam_b3lyp_6_31_plus_gdp_NICS.update({
    **cam_b3lyp_diffuse,
    '!coords_from' : '../orca_prepro_test',
    '!xyz_file'    : 'orca_prepro_test.xyz',
    '!overwrite' : True, 
})

#jobs are starting out of turn...

#THIS SHOULD BE ABLE TO BE DONE IN A SHORT TIME!
#SUPER SMALL MOLECULES, SUPER FAST METHODS.

xtb_opt_freq = {
        'program' : 'XTB',
        'functional' : 'gfn2',
        'run_type' : 'ohess', #opt-freq
        'xyz_file' : 'crest_best.xyz', #THIS DOESN'T WORK!
        #HAVE TO DO THIS FOR NOW!
        '!coords_from' : '../crest_gfn2__gfnff_squick_test',
        '!xyz_file' : 'crest_best.xyz',
    }

crude_confsearch_optfreq = {
    'crest_gfn2__gfnff_squick' : crest_gfn2__gfnff_squick,
    'xtb_gfn2_opt_freq' : xtb_opt_freq,
}
crude_property_calcs = { #here we assign filenames to each group of settings
    'r2scan_3c_triplet_sp_orbitals' : r2scan_3c_triplet_sp,
    'r2scan_3c_bs_singlet_sp_orbitals_occs' : r2scan_3c_singlet_sp,
    'r2scan_3c_tddft' : r2scan_3c_tddft,
    'orca_prepro' : orca_nics_prepro,
    'cam-b3ylp_6-31Gdp_NICS' : cam_b3lyp_6_31_plus_gdp_NICS,
}

gaussian_calcs = {
    'gaussian_prepro' : gauss_nics_prepro,
    'cam-b3ylp_6-31Gdp_NICS_gaussian' : cam_b3lyp_6_31_plus_gdp_NICS_gaussian,
}

#WHAT I'VE LEARNED...
#REALLY NEED TO MAKE TEST CALCULATIONS A PART OF THE
#WORK FLOW. STOP TESTING IN PRODUCTION
#JUST NO TIME. HAVE TO KEEP GAMBLING TO GET AHEAD.

In [23]:
croot_dir ='/gpfs/research/alabuginlab/gage/michael/test/'

test_settings = { 'test' : {
    'runtime' : '0-00:10:00',
    }
}

input_combi.do_everything(
     [global_configs,xtb_cm_states,mike_mols,crude_confsearch_optfreq,test_settings],
     croot_dir,
)

input_combi.do_everything(
    [global_configs,orca_cm_states,mike_mols,crude_property_calcs,test_settings],
    croot_dir,
)
input_combi.do_everything(
    [global_configs,gaussian_cm_states,mike_mols,gaussian_calcs,test_settings],
    croot_dir,
)

Error copying file: [Errno 2] No such file or directory: '/gpfs/home/gdb20/code/mol-maker/data/mike/nicstest/crest_best.xyz'
Error copying file: [Errno 2] No such file or directory: '/gpfs/home/gdb20/code/mol-maker/data/mike/nicstest/crest_best.xyz'
Error copying file: [Errno 2] No such file or directory: '/gpfs/home/gdb20/code/mol-maker/data/mike/nicstest/crest_best.xyz'
Error copying file: [Errno 2] No such file or directory: '/gpfs/home/gdb20/code/mol-maker/data/mike/nicstest/crest_best.xyz'
Error copying file: [Errno 2] No such file or directory: '/gpfs/home/gdb20/code/mol-maker/data/mike/nicstest/xtbopt.xyz'
Error copying file: [Errno 2] No such file or directory: '/gpfs/home/gdb20/code/mol-maker/data/mike/nicstest/xtbopt.xyz'
Error copying file: [Errno 2] No such file or directory: '/gpfs/home/gdb20/code/mol-maker/data/mike/nicstest/xtbopt.xyz'
Error copying file: [Errno 2] No such file or directory: '/gpfs/home/gdb20/code/mol-maker/data/mike/nicstest/xtbopt.xyz'
Error copying fi